# Initialization of libraries

In [1]:
import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# Our predefined dataset with questions we will load into our models

In [5]:
pr_qst = pd.DataFrame([ ## dataset
    {"question": "What is the historical significance of the mosaic in the apse of the Church of Panagia Aggeloktisti and in what century was it created?",
     "answer": "The mosaic in the apse of the Church of Panagia Aggeloktisti is of significant historical value as it is one of the finest examples of early Byzantine art in Cyprus. It depicts the Virgin Mary holding the Child Christ, flanked by archangels, and reflects the strong influence of Constantinopolitan art. It was created in the 6th century AD, making it a rare and invaluable work of early Christian iconography.",
     "model_response": "",
     },
    {"question": "What architectural features distinguish the Church of Panagia Aggeloktisti and what periods of construction does it unite?",
     "answer": "The Church of Panagia Aggeloktisti combines elements of a 6th-century basilica with a later, 11th-century domed cross-in-square church. Key architectural features include the preserved apse and mosaic from the early basilica, and the elegant domed structure added during the Middle Byzantine period, reflecting a synthesis of early Christian and Byzantine architecture.",
     "model_response": "",
     },
    {"question": "What does the name Aggeloktisti mean and where does it come from?",
     "answer": "The name 'Aggeloktisti' means 'built by angels' in Greek. According to local legend, angels constructed the church overnight after the destruction of an earlier Christian basilica, giving rise to the belief that it was divinely rebuilt.",
     "model_response": "",
     },
    {"question": "What chapels were added to the church in the Middle Ages and what was their purpose?",
     "answer": "In the Middle Ages, several chapels were added to the church complex, including a north chapel and others dedicated to saints. These chapels served both liturgical and funerary purposes, as well as providing space for local religious devotion and pilgrimage practices.",
     "model_response": "",
     },
    {"question": "How does the Church of Panagia Aggeloktisti demonstrate the cultural and artistic links between different regions of the Mediterranean?",
     "answer": "The church demonstrates cultural and artistic connections across the Mediterranean through its mosaic, which shows strong affinities with Constantinopolitan art, and its architecture, which blends Eastern Orthodox and local Cypriot styles. It stands as a testament to Cyprus’s role as a crossroads of civilizations, influenced by Byzantine, Greek, and Levantine traditions.",
     "model_response": "",
     },
    {"question": "What arguments are put forward for the inclusion of the Church of Panagia Aggeloktisti in the UNESCO World Heritage List under criterion (i)?",
     "answer": "The Church of Panagia Aggeloktisti is considered for inclusion under UNESCO criterion (i) because it represents a masterpiece of human creative genius, particularly due to its exceptional 6th-century mosaic. This work exemplifies the artistic and theological sophistication of early Christian art in the Byzantine world.",
     "model_response": "",
     },
    {"question": "What conservation measures have been taken since 1952 to preserve the mosaics and other elements of the church?",
     "answer": "Since 1952, conservation measures have included structural repairs, restoration of the dome and roofing, cleaning and stabilization of the mosaic, and ongoing archaeological and architectural studies. These efforts have aimed to protect the site’s artistic and historical integrity while ensuring its continued use and accessibility.",
     "model_response": "",
     },
    {"question": "How does the church reflect the religious and spiritual life of the local community over the centuries?",
     "answer": "The church reflects the religious and spiritual life of the local community through its continuous use as a place of worship, its role in local legends, and its integration into village life through feasts and traditions associated with the Virgin Mary. It has served as a spiritual center for centuries, adapting to the community’s evolving needs.",
     "model_response": "",
    },
    {"question": "How does the church of Panagia Aggeloktisti maintain its authenticity and integrity despite the passing centuries and the restorations carried out?",
     "answer": "Despite various restorations, the church retains a high degree of authenticity through the preservation of original elements like the apse mosaic and early Christian layout. Careful restoration practices have maintained its historical fabric, while the unbroken tradition of religious use has ensured the integrity of its spiritual and communal functions.",
     "model_response": "",
    }
])

# Models init

In [18]:
## llama

# llama = "openlm-research/open_llama_3b"
# llama_model = AutoModelForCausalLM.from_pretrained(
#     llama,
#     torch_dtype=torch.float16, ## gpu support
#     device_map="auto",
#     )
# llama_tokenizer = AutoTokenizer.from_pretrained(llama)


# tiny-llama
# tiny = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
# tiny_model = AutoModelForCausalLM.from_pretrained(
#     tiny,
#     torch_dtype=torch.float16, ## gpu support
#     device_map="auto",
#     )
# tiny_tokenizer = AutoTokenizer.from_pretrained(tiny)


## phi
# phi = "microsoft/phi-2"
# phi_model = AutoModelForCausalLM.from_pretrained(
#     phi,
#     torch_dtype=torch.float16, ## gpu support
#     device_map="auto",
#     trust_remote_code=True ## fix api issues
#     )
# phi_tokenizer = AutoTokenizer.from_pretrained(phi)

# falcon
falcon = "tiiuae/falcon-rw-1b"
falcon_model = AutoModelForCausalLM.from_pretrained(
    falcon,
    torch_dtype=torch.float16, ## gpu support
    device_map="auto",
    offload_folder="offload" ## memory issue
    )
falcon_tokenizer = AutoTokenizer.from_pretrained(falcon)


# Generate function to talk with LLM, responses to store output, output function to store in a list, after that save it in .csv file

In [16]:
def gen(question, model, tokenizer):
    prompt = f"Answer the following question clearly and completely:\n{question}\n"
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=300,
        num_return_sequences=1,
        do_sample=False, ## variety, turn off for now
        top_p=0.95,
        temperature=0.5, ## temp
        repetition_penalty=1.2,        # no dublicates
        no_repeat_ngram_size=4,
        pad_token_id=tokenizer.eos_token_id,
    )
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    if "Answer:" in answer:
        answer = answer.split("Answer:")[-1].strip()
    else:
        answer = answer.strip()

    return answer

# outputs
llama_responses = [] # 20 seconds seconds runtime (3b model)
falcon_responses = [] # 45 seconds (1b model)
phi_responses = [] # 20 seconds -> 100 max_length (2b? model)
tiny_responses = [] # 17 seconds -> 100 max_length (1b model)

# dataframes
llama_dataframe = pd.DataFrame()
falcon_dataframe = pd.DataFrame()
phi_dataframe = pd.DataFrame()
tiny_dataframe = pd.DataFrame()

def output(model, tokenizer, responses, dataframe):
    for question in pr_qst["question"]:
        resp = gen(question, model, tokenizer)
        responses.append(resp)
    dataframe = pr_qst.copy()
    dataframe["model_response"] = responses
    return dataframe

## llama
# llama_result = output(llama_model, llama_tokenizer, llama_responses, llama_dataframe)
# llama_result.to_csv("llama_results.csv")

## falcon
falcon_result = output(falcon_model, falcon_tokenizer, falcon_responses, falcon_dataframe)
falcon_result.to_csv("falcon_results.csv")

## phi
# phi_result = output(phi_model, phi_tokenizer, phi_responses, phi_dataframe)
# phi_result.to_csv("phi_results.csv")

## tiny llama
# result = output(tiny_model, tiny_tokenizer, tiny_responses, tiny_dataframe)
# result.to_csv("tiny_llama_results.csv")


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


In [17]:
falcon_result

,question,answer,model_response
0,What is the historical significance of the mos...,The mosaic in the apse of the Church of Panagi...,Answer the following question clearly and comp...
1,What architectural features distinguish the Ch...,The Church of Panagia Aggeloktisti combines el...,Answer the following question clearly and comp...
2,What does the name Aggeloktisti mean and where...,The name 'Aggeloktisti' means 'built by angels...,Answer the following question clearly and comp...
3,What chapels were added to the church in the M...,"In the Middle Ages, several chapels were added...",Answer the following question clearly and comp...
4,How does the Church of Panagia Aggeloktisti de...,The church demonstrates cultural and artistic ...,Answer the following question clearly and comp...
5,What arguments are put forward for the inclusi...,The Church of Panagia Aggeloktisti is consider...,Answer the following question clearly and comp...
6,What conservation measures have been taken sin...,"Since 1952, conservation measures have include...",Answer the following question clearly and comp...
7,How does the church reflect the religious and ...,The church reflects the religious and spiritua...,Answer the following question clearly and comp...
8,How does the church of Panagia Aggeloktisti ma...,"Despite various restorations, the church retai...",Answer the following question clearly and comp...
